In [1]:
import requests
import json

In [2]:
osmose_url= "http://osmose.openstreetmap.fr/en/api/0.3beta/issues?osm_type=relation&osm_id={}&full=true"

In [3]:
bbox_for_overpass = "5.173011321882331,-4.258575439453126,5.544913134097361,-3.572616577148438"

In [4]:
overpass_base_url = "http://overpass-api.de/api/interpreter?data="
overpass_query_part_for_bus_lines = '[out:json];relation["route"="bus"]({});relation["type"="route_master"](br);out meta;'.format(bbox_for_overpass)
overpass_query_part_for_bus_routes = '[out:json];relation["route"="bus"]({});out meta;'.format(bbox_for_overpass)

bus_lines_call = requests.get(overpass_base_url + overpass_query_part_for_bus_lines)
if bus_lines_call.status_code != 200:
    print("erreur overpass :{}".format(bus_lines_call.status_code))

bus_routes_call = requests.get(overpass_base_url + overpass_query_part_for_bus_routes)
if bus_routes_call.status_code != 200:
    print("erreur overpass :{}".format(bus_routes_call.status_code))

bus_routes = bus_routes_call.json()["elements"]
bus_lines = bus_lines_call.json()["elements"]
bus_relations = [line['id'] for line in bus_lines] + [route['id'] for route in bus_routes]

In [5]:
seems_ok = []
to_check = []

for a_relation in bus_relations:
    osmose_ = osmose_url.format(a_relation)
    osmose_call = requests.get(osmose_)
    osmose_results = osmose_call.json()['issues']
    if not osmose_results:
        seems_ok.append(a_relation)
    elif len(osmose_results) == 1:
        if osmose_results[0]['classs'] == 21404: #ignore "missing ref tag"
            seems_ok.append(a_relation)
        else :
            to_check.append(a_relation)
    else :
        to_check.append(a_relation)


In [6]:
seems_ok_objects = ["r{}".format(relation) for relation in seems_ok]
josm_ok = "http://localhost:8111/load_object?new_layer=true&relation_members=true&objects="+",".join(seems_ok_objects)
josm_ok

'http://localhost:8111/load_object?new_layer=true&relation_members=true&objects=r5985016,r9589447,r10063025,r10087961,r10168184,r10173635,r10179006,r10179435,r10184139,r10184730,r10184964,r10185142,r10185169,r10185403,r10185406,r10187611,r10187867,r10188869,r10189319,r10190159,r10190354,r10191924,r10192143,r10192153,r10199868,r10200077,r10201145,r10206110,r10206588,r10206969,r10206986,r10211372,r10211655,r10211824,r10212284,r10212293,r10212448,r10213167,r10213431,r10215245,r10215413,r10215677,r10216195,r10216288,r10217855,r10220180,r10220245,r10220807,r10225666,r5986717,r8771035,r8778229,r9589446,r10063024,r10087879,r10173633,r10173634,r10178996,r10179433,r10183939,r10184124,r10184409,r10184729,r10184954,r10184962,r10184963,r10185122,r10185123,r10185168,r10185402,r10185404,r10185405,r10186002,r10186314,r10187868,r10188094,r10188362,r10188868,r10189025,r10189317,r10189318,r10190158,r10190187,r10190353,r10191847,r10191923,r10191926,r10192050,r10192152,r10192285,r10192310,r10199196,r10199

In [7]:
to_check_objects = ["r{}".format(relation) for relation in to_check]
to_check_ok = "http://localhost:8111/load_object?new_layer=true&relation_members=true&objects="+",".join(to_check_objects)
to_check_ok

'http://localhost:8111/load_object?new_layer=true&relation_members=true&objects=r4153675,r5985015,r8774949,r8778704,r9589445,r10178669,r10179434,r10220806'